In [ ]:
import torchvision.models as models
import torchvision.datasets as datasets
import torch.nn as nn
import torch
import torch.optim as optim
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torchvision


In [ ]:
imagenet_data = torchvision.datasets.FGVCAircraft('path/to/fgvcaircraft_root/',download=True,split='trainval')
data_loader = torch.utils.data.DataLoader(imagenet_data,
                                          batch_size=4,
                                          shuffle=True,
)

In [ ]:
num_classes =100

In [ ]:
model  = models.GoogLeNet()

In [ ]:
num_ftrs = model.fc.in_features

In [ ]:
model.fc = nn.Linear(num_ftrs,num_classes)

In [ ]:
learning_rate = 1e-2
optimizer = optim.Adam(model.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()
batch_size = 64

In [ ]:
def train_loop(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred,y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]") 


def test_loop(dataloader,model,loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    with torch.no_grad():
        for (X,y) in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(data_loader, model, loss_fn, optimizer)
    test_loop(data_loader, model, loss_fn)
print("Done!")